In [48]:
import tensorflow as tf

# each row represents a user ['Vijay', 'Danielle', 'Ryan', 'Chris']
# each column represents a movie ['Star Wars', 'Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Harry Potter']

users_movies = tf.constant([[4, 6, 8, 0, 0],
              [0, 0, 10, 0, 8],
              [0, 6, 0, 0, 3],
              [10, 9, 0, 5, 0]])
# the columns represent ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([[1, 1, 0, 0, 1],
              [1, 1, 0, 0, 0],
              [0, 0, 1, 1, 0],
              [1, 0, 1, 1, 0],
              [0, 0, 0, 0, 1]])

In [49]:
users_movies

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[ 4,  6,  8,  0,  0],
       [ 0,  0, 10,  0,  8],
       [ 0,  6,  0,  0,  3],
       [10,  9,  0,  5,  0]], dtype=int32)>

In [50]:
movies_feats

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 1, 1, 0],
       [0, 0, 0, 0, 1]], dtype=int32)>

In [51]:
print(tf.expand_dims(tf.transpose(users_movies)[:,0], axis = 1))
tf.expand_dims(tf.transpose(users_movies)[:,0], axis = 1)* movies_feats

tf.Tensor(
[[4]
 [6]
 [8]
 [0]
 [0]], shape=(5, 1), dtype=int32)


<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[4, 4, 0, 0, 4],
       [6, 6, 0, 0, 0],
       [0, 0, 8, 8, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int32)>

In [52]:
## We first build a list of the weighted feature matrices for each user, then use tf.stack applied to this list, setting the stack axis to be zero. 
num_users = 4

wgtd_feature_matrices = [tf.expand_dims(tf.transpose(users_movies)[:,i], axis = 1) *movies_feats for i in range(num_users)]
wgtd_feature_matrices

[<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[4, 4, 0, 0, 4],
        [6, 6, 0, 0, 0],
        [0, 0, 8, 8, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0, 10, 10,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  8]], dtype=int32)>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0],
        [6, 6, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 3]], dtype=int32)>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[10, 10,  0,  0, 10],
        [ 9,  9,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 5,  0,  5,  5,  0],
        [ 0,  0,  0,  0,  0]], dtype=int32)>]

In [53]:
users_movies_feats = tf.stack(wgtd_feature_matrices, axis = 0)
users_movies_feats

<tf.Tensor: shape=(4, 5, 5), dtype=int32, numpy=
array([[[ 4,  4,  0,  0,  4],
        [ 6,  6,  0,  0,  0],
        [ 0,  0,  8,  8,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0, 10, 10,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  8]],

       [[ 0,  0,  0,  0,  0],
        [ 6,  6,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  3]],

       [[10, 10,  0,  0, 10],
        [ 9,  9,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 5,  0,  5,  5,  0],
        [ 0,  0,  0,  0,  0]]], dtype=int32)>

In [54]:
## Finding the user feature tensor
# 1st sum each column, each row represents the sum of the feature values for each user
users_movies_feats_sums = tf.reduce_sum(users_movies_feats, axis = 1)
users_movies_feats_sums

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[10, 10,  8,  8,  4],
       [ 0,  0, 10, 10,  8],
       [ 6,  6,  0,  0,  3],
       [24, 19,  5,  5, 10]], dtype=int32)>

In [55]:
# find total for each user, each row for each user
users_movies_feats_totals = tf.reduce_sum(users_movies_feats_sums, axis = 1)
users_movies_feats_totals

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([40, 28, 15, 63], dtype=int32)>

In [56]:
# normalization is then just the result of dividing feature sum by the feature totals for each user.
# In the end, stack the resulting tensors together to get final users features tensor
users_feats = tf.stack([users_movies_feats_sums[i,:]/users_movies_feats_totals[i] for i in range(num_users)], axis = 0)
# This results in a user feature tensor, where each row corresponds to a specific user feature vector
users_feats

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.        , 0.        , 0.35714286, 0.35714286, 0.28571429],
       [0.4       , 0.4       , 0.        , 0.        , 0.2       ],
       [0.38095238, 0.3015873 , 0.07936508, 0.07936508, 0.15873016]])>

In [57]:
# to find the inferred movie rankings for our users, we compute the dot product between each user feature vector and each movie feature vector.
# we will see how similar each user is with respect to each movie as measured across these five feature dimensions.

# use map function, it repeatedly apply some callable function on a sequence of elements from first to last.
# dot product of each user feature vector with each movie feature vector
# variable user ratings holds the list of the resulting movie ratings for each user and each movie
users_ratings = [tf.map_fn(lambda x: tf.tensordot(users_feats[i], x, axes = 1), tf.cast(movies_feats, tf.float64)) for i in range(num_users)]
users_ratings

[<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.6 , 0.5 , 0.4 , 0.65, 0.1 ])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.28571429, 0.        , 0.71428571, 0.71428571, 0.28571429])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([1. , 0.8, 0. , 0.4, 0.2])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.84126984, 0.68253968, 0.15873016, 0.53968254, 0.15873016])>]

Once we had the user-movie ranking matrix, we can compare it with the original user-movie matrix to see which movies to recommend to which user. Because our users have already seen and rated some of our movies, we want to mask the ranking for previously rated movies and focus only on unrated or unseen movies for each user.

In [58]:
# stack them together to get a tensor of all the users and their movie ratings
all_users_ratings = tf.stack(users_ratings)
all_users_ratings

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ],
       [0.28571429, 0.        , 0.71428571, 0.71428571, 0.28571429],
       [1.        , 0.8       , 0.        , 0.4       , 0.2       ],
       [0.84126984, 0.68253968, 0.15873016, 0.53968254, 0.15873016]])>

One of drawbacks of content-based recommender systems. It can be difficult to expand the interest of a user. If the user hasn't rated movies within one of our predefined features, our recommender won't suggest new movies in that genre. In this sense, content-based recommenders aren't good at expanding the interest of users to new domains.